In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



Using TensorFlow backend.


In [2]:
import telegram
from keras.callbacks import Callback
from callbacks import TelegramCallback
#from callbacks.TelegramData import TelegramData


# create callback
config = {
    'token': "758990540:AAFrdfPpNGctKEvoMvMm3vFpj_VwY_BeOA4",   # paste your bot token
    'telegram_id': 158106307,                                   # paste your telegram_id
}

tg_callback = TelegramCallback(config)

In [3]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text ("Run_Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)

In [4]:
dfAll = pd.read_pickle("PklData/df_blobs.pkl")

In [5]:
dfAll.userID.unique()

array([ 3,  7,  5,  9, 14,  6, 10,  2,  4, 12,  1, 13,  8, 11])

In [35]:
# the data, split between train and test sets

def createDataSets(train_ids, test_ids):

    df_train = dfAll[dfAll.userID.isin(train_ids)]
    df_test = dfAll[dfAll.userID.isin(test_ids)]

    df_test = df_test.reset_index()
    df_train = df_train.reset_index()

    df_train2 = df_train[['Blobs', 'InputMethod']].copy()
    df_test2 = df_test[['Blobs', 'InputMethod']].copy()

    x_train = np.vstack(df_train2.Blobs)
    x_test = np.vstack(df_test2.Blobs)
    y_train = df_train2.InputMethod.values
    y_test = df_test2.InputMethod.values

    x_train = x_train.reshape(-1, 27, 15, 1)
    x_test = x_test.reshape(-1, 27, 15, 1)

    num_classes = 2
    y_train_one_hot = utils.to_categorical(df_train2.InputMethod, num_classes)
    y_test_one_hot = utils.to_categorical(df_test2.InputMethod, num_classes)

In [36]:
train_ids = [1,2,3, 7, 8, 9, 10,12,13,14]
test_ids = [4,5,6,11]
createDataSets(train_ids, test_ids)

In [8]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [48]:
def cnn_model():
    with(tf.device("/gpu:0")):
        optimizer = optimizers.Adam(lr=0.001)
        #optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
        #init=tf.global_variables_initializer()

        model = Sequential()
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(27,15,1)))
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
        model.add(BatchNormalization(axis=-1))
        model.add(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last'))
        model.add(Dropout(0.50))

        model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
        model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
        model.add(BatchNormalization(axis=-1))
        model.add(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last'))
        model.add(Dropout(0.50))

        model.add(Flatten())
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.L1L2(0.02, 0.15), use_bias=True))
        model.add(Dropout(0.50))
        model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.L1L2(0.02, 0.15), use_bias=True))
        model.add(Dropout(0.50))
        model.add(Dense(num_classes, activation='softmax'))

        ####TENSORBOARD
        config = ""
        for layer in model.layers:
            config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
        config += "batchsize: " + str(batch_size) + "\n\n" + "epochs: " + str(epochs) + "\n\n"
        #### END TENSORBOARD

        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [49]:
tf.get_default_graph()

readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_Jan_" + readable_timestamp

#set early stopping criteria
pat = 5 #this is the number of epochs with no improvment after which the training will stop

logger = LoggingTensorBoard(settings_str_to_log = config, log_dir=tensorflowfolder, histogram_freq=0, write_graph=True, write_images=True, update_freq = 'epoch')
model_checkpoint = ModelCheckpoint("./ModelSnapshots/KnuckleFinger_Jan_CrossValidation" + readable_timestamp + ".h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.95, 
                                            min_lr=0.00001)

def fit_and_evaluate(EPOCHS=500, BATCH_SIZE=500):
    model = None
    model = cnn_model()
    model.summary()
    history = model.fit(x_train, y_train_one_hot,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(x_test, y_test_one_hot),
                    callbacks=[model_checkpoint, tg_callback, learning_rate_reduction])
    
    print("Val Score: ", model.evaluate(val_x, val_y))
    return history

In [ ]:
n_folds=len(dfAll.userID.unique())
epochs=500
batch_size=200

#save the model history in a list after fitting so that we can plot later
model_history = [] 

for i in range(n_folds):
    test_ids = [np.array(dfAll.userID.unique()[i]).tolist()]
    train_ids = np.delete(np.array(dfAll.userID.unique()), i).tolist()
     
    print("Training on Fold: ",i+1, "\n test_ids: ", test_ids, "\n train_ids: ", train_ids)
    createDataSets(test_ids, train_ids)
    #t_x, val_x, t_y, val_y = train_test_split(x_train, y_train, test_size=0.1, random_state = np.random.randint(1,1000, 1)[0])
    ####END
    model_history.append(fit_and_evaluate(epochs, batch_size))
    print("======="*12, end="\n\n\n")

Training on Fold:  1 
 test_ids:  [3] 
 train_ids:  [7, 5, 9, 14, 6, 10, 2, 4, 12, 1, 13, 8, 11]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 27, 15, 64)        640       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 27, 15, 64)        36928     
_________________________________________________________________
batch_normalization_7 (Batch (None, 27, 15, 64)        256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 8, 64)         0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 14, 8, 64)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 14, 8, 32)         18464     
_____________________________________________

In [39]:
# use model for inference to get test accuracy
y_test_pred = model.predict(x_test)
y_test_pred = np.argmax(y_test_pred, axis=1)

print ('\n Summary of the precision, recall, F1 score for each class:')
print (sklearn.metrics.classification_report(y_test, y_test_pred))

print ('\n Confusion matrix: ')
print (sklearn.metrics.confusion_matrix(y_test, y_test_pred))

[3]